# 🚀 Estrategia Jueves: Maximización de Eficiencia Robusta ("Plan C")

## 1. Diagnóstico y Justificación Táctica
Tras analizar el comportamiento del mercado, hemos identificado un régimen de **"Amplitud de Mercado Estrecha"**. La evidencia muestra que la mayoría de los activos aportan ruido y volatilidad, mientras que el *Alpha* (retorno real) se concentra en muy pocos nombres.

* **La Decisión:** Implementamos una estrategia de **Optimización en Dos Etapas**. En lugar de forzar una diversificación ingenua, utilizamos un algoritmo de filtrado previo para aislar la señal de calidad antes de asignar los pesos finales.

## 2. Metodología: Filtrado Algorítmico de Dos Etapas

Para eliminar el ruido estadístico inherente a la optimización sobre 50 activos con un histórico limitado, el código ejecuta un protocolo de selección automática basado en la **Unión de Subconjuntos Óptimos** (*Union of Optimal Subsets*).

El algoritmo realiza dos optimizaciones previas independientes para construir un universo de inversión depurado ($\mathbb{S}$):

### ETAPA 1: Filtro de Estabilidad (Componente Defensivo)
El algoritmo busca los activos que minimizan el riesgo estructural de la cartera, resolviendo el problema de **Mínima Varianza Global (GMV)**:

$$
\begin{aligned}
& \underset{w}{\text{minimizar}} & & \sigma_p^2 = w^T \Sigma w \\
& \text{sujeto a:} & & \sum_{i=1}^{N} w_i = 1 \\
& & & w_i \ge 0
\end{aligned}
$$

* **Criterio de Selección:** Se retienen automáticamente todos los activos cuyo peso en la solución óptima sea significativo ($w_i > 1\%$). Estos activos (ej. Activos 23 y 8) actúan como "anclas" de baja volatilidad y descorrelación.

### ETAPA 2: Filtro de Potencia (Componente de Alpha)
El algoritmo busca los activos con la mayor eficiencia marginal, resolviendo una optimización de **Máximo Ratio Sharpe sin restricciones** (utilizando la transformación convexa estándar):

$$
\begin{aligned}
& \underset{y}{\text{maximizar}} & & y^T \mu \\
& \text{sujeto a:} & & y^T \Sigma y \le 1 \\
& & & y_i \ge 0
\end{aligned}
$$
*(Donde los pesos finales se normalizan como $w = y / \sum y_i$)*

* **Criterio de Selección:** Los activos que sobreviven con peso positivo en esta cartera tangente teórica (ej. Activos 6 y 36) son seleccionados como los "motores de retorno" debido a su alto *Alpha* ajustado por riesgo.

---

**Resultado de la Selección Automática:**
El algoritmo define el subconjunto final $\mathbb{S}$ como la unión de los supervivientes de ambas etapas ($\mathbb{S} = \mathbb{S}_{Defensa} \cup \mathbb{S}_{Ataque}$), descartando automáticamente 43 activos ruidosos y reteniendo la siguiente estructura híbrida:

| Activo Detectado | Rol Asignado por el Algoritmo | Razón de Selección Automática |
| :--- | :--- | :--- |
| **Activo 6 y 36** | ⚔️ **Motores de Retorno** | Alta Eficiencia Marginal detectada en la Etapa 2 (Max Sharpe). |
| **Activo 23 y 8** | 🛡️ **Estabilizadores** | Baja varianza y correlación detectada en la Etapa 1 (MinVar). |

## 3. Formulación Matemática Final

Sobre este subconjunto depurado automáticamente ($\mathbb{S}$), se ejecuta la optimización final.

### A. Función Objetivo
Maximización de la Utilidad Media-Varianza sobre los activos seleccionados:

$$
\underset{w}{\text{maximizar }} \quad U(w) = w^T \mu - \gamma (w^T \Sigma w)
$$

### B. Restricciones del Modelo
El algoritmo aplica las restricciones finales sobre los supervivientes para garantizar el perfil de riesgo deseado:

1.  **Exposición Táctica (90%):**
    $$\sum_{i \in \mathbb{S}} w_i = 0.90$$
    > *Justificación:* Se maximiza la exposición a los activos de alta calidad detectados, minimizando el lastre del *Cash* (0% retorno).

2.  **Límite de Seguridad (30%):**
    $$0 \le w_i \le 0.30 \quad \forall i$$
    > *Justificación:* Se impone un techo duro ("Hard Cap") para evitar que la optimización matemática concentre el 100% en un solo activo, protegiendo la cartera contra eventos de riesgo idiosincrático.

In [6]:
import pandas as pd
import numpy as np
import cvxpy as cp

def optimizar_estrategia_final_30_percent():
    print("--- ESTRATEGIA FINAL: FRANCOTIRADOR EQUILIBRADO (30/30/30) ---")
    
    # 1. Cargar Datos
    try:
        df = pd.read_csv('prod_long_sharpe_u50_20260116_v5_train_dataset.csv')
    except:
        print("Error: No se encuentra el archivo .csv")
        return

    returns = df.values
    assets = df.columns
    
    # 2. Selección de Élite (Unión Martes + Jueves)
    indices_elite = [5, 35, 22, 7, 44, 4, 19]
    
    returns_sub = returns[:, indices_elite]
    mu = np.mean(returns_sub, axis=0)
    Sigma = np.cov(returns_sub, rowvar=False)
    
    # 3. Optimización
    w = cp.Variable(len(indices_elite))
    risk = cp.quad_form(w, Sigma)
    ret = mu @ w
    gamma = cp.Parameter(nonneg=True)
    
    # --- REGLAS DE ORO ---
    # 1. Inversión: 90% (Dejamos 10% en Cash por seguridad máxima)
    # 2. Concentración: Max 30% (Obliga a repartir entre 3 activos fuertes)
    prob = cp.Problem(cp.Maximize(ret - gamma * risk), 
                      [cp.sum(w) == 0.90, 
                       w >= 0, 
                       w <= 0.30]) 
    
    best_sharpe = -np.inf
    best_w = None
    
    # Barrido de Gamma para encontrar el punto óptimo
    for g_val in [0.5, 1.0, 2.0, 5.0, 10.0]:
        gamma.value = g_val
        try:
            prob.solve(solver=cp.SCS)
            if w.value is not None:
                curr_w = w.value
                val_vol = np.sqrt(curr_w.T @ Sigma @ curr_w)
                if val_vol > 1e-4:
                    sh = (curr_w @ mu) / val_vol
                    if sh > best_sharpe:
                        best_sharpe = sh
                        best_w = curr_w
        except: pass
    
    if best_w is None: return
    
    # 4. Procesar Salida (Limpieza de decimales)
    top_indices_local = np.argsort(best_w)[::-1]
    
    pesos_finales = np.zeros(len(assets))
    
    print("\n--- CARTERA FINAL SELECCIONADA ---")
    count = 0
    w_sum = 0
    
    for i in top_indices_local:
        peso = best_w[i]
        idx_global = indices_elite[i]
        
        # Filtro de limpieza (quitamos residuos < 1%)
        if peso > 0.01:
            if count < 5: # Asegurar Max 5 Activos
                pesos_finales[idx_global] = peso
                print(f"[{count+1}] Activo {assets[idx_global]} (Idx {idx_global}): {peso:.2%}")
                w_sum += peso
                count += 1
            else:
                # Si hubiera un 6º activo, lo ignoramos por reglas del concurso
                pass
    
    # Re-ajuste fino para que sume exactamente 0.90 (por si hemos borrado algún residual)
    pesos_finales = pesos_finales / np.sum(pesos_finales) * 0.90
    
    cadena = " ".join(["0.0" if p < 0.001 else str(round(p, 3)) for p in pesos_finales])
    
    print(f"\nTotal Invertido: {np.sum(pesos_finales):.1%}")
    print(f"Total Cash (Renta Fija): {(1-np.sum(pesos_finales)):.1%}")
    print("\n>>> COPIA ESTA CADENA <<<")
    print(cadena)

if __name__ == "__main__":
    optimizar_estrategia_final_30_percent()

--- ESTRATEGIA FINAL: FRANCOTIRADOR EQUILIBRADO (30/30/30) ---

--- CARTERA FINAL SELECCIONADA ---
[1] Activo asset36 (Idx 35): 30.00%
[2] Activo asset6 (Idx 5): 30.00%
[3] Activo asset23 (Idx 22): 19.38%
[4] Activo asset8 (Idx 7): 10.61%

Total Invertido: 90.0%
Total Cash (Renta Fija): 10.0%

>>> COPIA ESTA CADENA <<<
0.0 0.0 0.0 0.0 0.0 0.3 0.0 0.106 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.194 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.3 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
